In [1]:
from pathlib import Path
import os

yr = 2023
d = 13

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
import numpy as np

def format_input(inp):
  cur_pattern = []
  patterns = []
  for l in inp.split('\n'):
    if len(l) != 0:
      cur_pattern.append(list(l.replace('.', '0').replace('#', '1')))
    else:
      patterns.append(np.array(cur_pattern, dtype=int))
      cur_pattern = []
  patterns.append(np.array(cur_pattern, dtype=int))
  return patterns

In [3]:
def check_for_symmetry(pattern, index=1, how='vertical', smudged=False):

  if how=='horizontal':
    h1 = pattern.T[:,:index]
    h2 = pattern.T[:,index:]
  elif how=='vertical':
    h1 = pattern[:,:index]
    h2 = pattern[:,index:]

  if h1.shape[1] > h2.shape[1]:
    h1 = h1[:,h1.shape[1]-h2.shape[1]:]
  elif h1.shape[1] < h2.shape[1]:
    h2 = h2[:,:-(h2.shape[1]-h1.shape[1])]

  if not smudged:
    return np.all(h1[:,::-1] == h2)
  else:
    # If we're one off from a symmetry, we found a smudge
    return np.sum((h1[:,::-1] != h2).flatten().astype(int))==1

In [4]:
def find_all_symmetries(pattern, how='vertical', smudged=False):
  if how=='vertical':
    r = pattern.shape[1]
  elif how=='horizontal':
    r = pattern.shape[0]

  symmetries = []
  for i in range(1, r):
    if check_for_symmetry(pattern, i, how, smudged):
      symmetries.append(i)
  return symmetries

In [5]:
def sum_of_symmetries(formatted_input, smudged=False):
  s = 0
  for pattern in formatted_input:
    hsym = find_all_symmetries(pattern, 'horizontal', smudged)
    vsym = find_all_symmetries(pattern, 'vertical', smudged)
    s += (sum(hsym)*100) + sum(vsym)
  return s

In [6]:
import time

t = time.time()

formatted_input = format_input(inp)

print(sum_of_symmetries(formatted_input))
print(sum_of_symmetries(formatted_input, smudged=True))

print('\nRUNTIME: ', time.time()-t)

33356
28475

RUNTIME:  0.035903215408325195
